In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


from categorizar import *
from demand_model import *
from relatorio import *

## Base Para escorar e otimizar

In [2]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Data Table - Otimizacao segmentacao 49.7.csv",
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

df_lote_102024["premio"] = df_lote_102024["Premio_oferecido_com_batente"]

In [3]:
df_lote_escorar=df_lote_102024.copy()

In [4]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [5]:
df_lote = categorizar(df_lote_escorar)

In [6]:
df_lote.shape

(1459, 145)

In [7]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [8]:
df_lote.drop(['Demanda_basecase', 'Renewal Demand','Premium Parameter'],axis='columns', inplace=True)

In [ ]:
#df_lote.columns.to_list()

In [9]:
def demand_model(df):
    import pandas as pd
    import numpy as np 
    P   = df['premio']
    X2  = np.array(df['GBM_mercado'])
    X3  = np.array(df['Premio_IC_100'])
    X4  = np.array(df['premio_anterior'])
    X5  = np.array(df['MD_tempo_casa_2'])
    X6  = np.array(df['per_com'])
    X7  = np.array(df['per_com_bin_p_1_10'])
    X1  = np.array(df['is_casco_ofer'])
    X8  = np.array(df['canal_bin_Corretor_Mais'])
    X9  = np.array(df['Cod_End_bin_13509'])
    X10 = np.array(df['fator_ajuste_bin_f_101_120'])
    X11 = np.array(df['classe_bonus_bin_00'])
    X12 = np.array(df['UF_bin_AL_CE_RS'])
    X13 = np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14 = np.array(df['UF_bin_GO_MA_MG'])
    X15 = np.array(df['UF_bin_MT_PR'])
    X16 = np.array(df['UF_bin_RJ'])
    X17 = np.array(df['MD_comb_bin_Gasolina'])
    X18 = np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19 = np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20 = np.array(df['fx_ir_12m_bin_01_00_20'])
    X21 = np.array(df['fx_ir_12m_bin_03_40_60'])
    X22 = np.array(df['fx_ir_12m_bin_04_60_80'])
    X23 = np.array(df['fx_ir_12m_bin_05_80_100'])
    X24 = np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25 = np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26 = np.array(df['idade_ult_RNS_bin_06_inf'])

    def demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)


    df['Renewal_Demand'] = demanda(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)
    
    return df

In [11]:
df_lote = demand_model(df_lote)

In [12]:
np.mean(df_lote['Renewal_Demand'])

0.5073676747519267

In [13]:
def relatorio(df):
    import pandas as pd
    import numpy as np 
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["Renewal_Demand"]*df["premio"]
    df["premio_anterior_prob"] = df["Renewal_Demand"]*df["premio_anterior"]
    df["Act_cost_prob"] = df["Renewal_Demand"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["premio"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]



    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["Renewal_Demand"]),0)
    premio_ofertado     = np.mean(df["premio"])
    retencao_estimada   = np.mean(df["Renewal_Demand"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["Renewal_Demand"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["Renewal_Demand"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["premio"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["premio"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df_lote["premio_anterior"])
    premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["Renewal_Demand"])


    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado      
                            ])


    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:']

    return tabela

In [14]:
relatorio(df_lote)

,Total
Apolices Ofertadas:,1459.000000
Apolices Renovadas:,740.000000
%Retenção:,0.507368
Premio Ofertado:,2993.303856
Premio Renovado:,2388.436295
Premio Anterior Ofertado:,2863.094414
Premio Anterior Renovado:,2634.386801
Risco Ofertado,1726.892590
Risco Renovado,1347.662024
%Comissão Ofertada:,0.155935


## Otimização Teste 

In [15]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(1459, 180)

In [16]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(1459, 180)

In [24]:
import sympy as sp
import numpy as np

# Definindo as variáveis
P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, C, Lambda, K = sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26 C Lambda K')

# Definindo a função F
F = sp.exp(1.3356 - 0.2576 * sp.log(P) + 1.3647 * (P / X1) - 0.8782 * sp.log(P / X2) - 0.795 * sp.log(P / X3) 
           - 1.6731 * sp.log(P / X4) + 0.3797 * sp.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 
           - 0.1382 * X9 + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 - 0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 
           + 0.2817 * X16 + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 + 0.4343 * X21 + 1.1066 * X22 
           + 1.9561 * X23 - 0.0742 * X24 - 0.8459 * X25 - 0.7094 * X26) * (P - C) - Lambda * (
        sp.exp(1.3356 - 0.2576 * sp.log(P) + 1.3647 * (P / X1) - 0.8782 * sp.log(P / X2) - 0.795 * sp.log(P / X3) 
             - 1.6731 * sp.log(P / X4) + 0.3797 * sp.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 - 0.1382 * X9 
             + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 - 0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 + 0.2817 * X16 
             + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 + 0.4343 * X21 + 1.1066 * X22 + 1.9561 * X23 - 0.0742 * X24 
                - 0.8459 * X25 - 0.7094 * X26) - K)

# Derivando F em relação a P
dF_dP = sp.diff(F, P)

# Derivando F em relação a Lambda
dF_dLambda = sp.diff(F, Lambda)

# Convertendo as derivadas em funções utilizáveis
dF_dP_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, C, Lambda, K), dF_dP, 'numpy')
dF_dLambda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, C, Lambda, K), dF_dLambda, 'numpy')

# Exemplo de uso das funções
P_val = 1.0
X_vals = [1.0] * 26
C_val = 1.0
Lambda_val = 1.0
K_val = 1.0

dF_dP_result = dF_dP_func(P_val, *X_vals, C_val, Lambda_val, K_val)
dF_dLambda_result = dF_dLambda_func(P_val, *X_vals, C_val, Lambda_val, K_val)

In [26]:
dF_dLambda_result

-55.39542196965049

In [27]:
dF_dP_result

182.67605084409124

#########################################

In [ ]:
#[x for x in df.columns if "com_bin" in x]

In [28]:
%%time
######
import sympy as sp
from scipy.optimize import fsolve

df = df_lote_otim

target_demand=0.525

fator_retencao = np.mean(df['Demanda_base'])/target_demand

df['fator_retencao']=fator_retencao

df["IC_ofer_python0"] = df["per_com_pct"] + df["Act_cost"]/df["premio"] +  0.2749

for index, row in df.iterrows():
    if row['IC_ofer_python0']   <= 0.8:
        df.at[index, 'fat_ajuste_otim0'] = 0.95
    elif row['IC_ofer_python0']   <= 0.95:
        df.at[index, 'fat_ajuste_otim0'] = 0.92        
    elif row['IC_ofer_python0']   <= 0.99:
        df.at[index, 'fat_ajuste_otim0'] = 0.99

    elif row['IC_ofer_python0']   >= 1.01:
        df.at[index, 'fat_ajuste_otim0'] = 1.02
    elif row['IC_ofer_python0']   >= 1.03:
        df.at[index, 'fat_ajuste_otim0'] = 1.04
    else:
        df.at[index, 'fat_ajuste_otim0'] = 1.0
    

def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(P):
    
    #df['precos']= P *df["fat_ajuste_otim0"] 
    df['precos']= P *1.23
    
    
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
    
    # Calcular a margem esperada
    df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['precos'] - df['Act_cost'] - df['per_com']/100 - 0.2749*df['precos'])
    
  
    margem_esperada_total = df['margem_esperada_otimizada'].sum() 

    # Maximizar a margem 
    return -margem_esperada_total

# Definir os limites para os preços otimizado
nome_variaveis = ['LB','UB']
limites=np.array(df.loc[:,nome_variaveis])

# Executar a otimização 
resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11,seed=df.shape[0])

# Resultados
print(resultado.message) 
print("Tamanho da Base:", df.shape[0])
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun
margem_esperada_total_otimizada = -resultado.fun


Optimization terminated successfully.
Tamanho da Base: 1459
CPU times: total: 1min 5s
Wall time: 1min 11s


In [26]:
print(df['precos'].mean()),
print(np.mean(precos_otimizados)),
print(np.mean(-resultado.fun))

3837.610510627989
3004.33343043582
587191.2182896801


In [ ]:
#df3.to_csv(r'df3_avalia_prem.csv', sep='|' ,header='true')

In [24]:
df['Demanda_base'] = df['Renewal_Demand']



df['batente_otim_min']=0.9
df['batente_otim_max']=1.1
df['batente_otim_prem_ant_min']=0.5
df['batente_otim_prem_ant_max']=1.8

 
df['precos_otimizados'] = precos_otimizados


df['retencao_otimizada'] = calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['margem_esperada_base_case'] = df['Demanda_base']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()


df['fat_teste0']= df['precos_otimizados']/df['premio']
df['fat_teste1']=df['precos_otimizados']/df['premio']-1 

fat_teste0= df['fat_teste0'].values
fat_teste1= df['fat_teste1'].values

if (fat_teste0 > 1).any() ==True:
    fat_teste = fat_teste1
else:
    fat_teste=0
    
df['fat_teste'] =  fat_teste
    
    
delta_q = (df['retencao_otimizada']-df['Demanda_base'])
delta_p = (df['precos_otimizados']-df['premio'])
   
delta_q_b = ((df['retencao_otimizada']+df['Demanda_base'])/2)
delta_p_b = ((df['precos_otimizados']+df['premio'])/2)

df["elasticidade"] = (delta_q/delta_p)*(delta_p_b/delta_q_b)
df["elasticidade_2"] = (df['retencao_otimizada']/df['Demanda_base']-1)/(df['precos_otimizados']/df['premio']-1)  

df["elasticidade"] = df["elasticidade"].fillna(-1)
df["elasticidade_2"] = df["elasticidade_2"].fillna(-1)

df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.5,-2.5,-2,-1.5,-1,0])  

df.replace([np.inf, -np.inf], -1, inplace=True)



df["SP_ofer_python"] = df["Act_cost"]/df["precos_otimizados"]
df["IC_ofer_python"] = df["per_com_pct"] + df["SP_ofer_python"] +  0.2749
#df["SP_ofer_eranix"] = df["Act_cost"]/df["Premium Parameter"]
#df["IC_ofer_eranix"] = df["SP_ofer_eranix"] + df["per_com_pct"]+  0.2749
df["varia_prem_anterior_py"] = df["precos_otimizados"]/df["premio_anterior"]
#df["varia_prem_anterior_earnix"] = df["Premium Parameter"]/df["premio_anterior"]
df["varia_prem_ofer_py"] = df["precos_otimizados"]/df["premio"]
#df["varia_prem_ofer_earnix"] = df["Premium Parameter"]/df["premio"]
#df["varia_prem_py_earnix"] = df["precos_otimizados"]/df["Premium Parameter"]


df['IC_ofer_python_bin'] = pd.cut(x=df['IC_ofer_python'], bins=[0.7,0.95,0.99,1,1.03,1.9])  
#df['IC_ofer_eranix'] = pd.cut(x=df['IC_ofer_eranix'], bins=[0.7,0.95,0.99,1,1.03,1.9]) 

df['fat_ajuste_otim'] = 1.0  

for index, row in df.iterrows():
    if row['IC_ofer_python'] <= 0.95:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.88
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.97
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.015
            
    elif row['IC_ofer_python']<= 0.99:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.99
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.96
            
             
    elif row['IC_ofer_python']<= 1:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1      
            
            
            
    elif row['IC_ofer_python']<= 1.03:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1            
            
    elif row['IC_ofer_python']<= 3:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.02
            
            
#df['fat_ajuste_otim']=fat_ajuste_otim

# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Base Case:", df['Demanda_base'].mean())
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Elasticidade:", df['elasticidade'].mean())
print("Elasticidade2:", df['elasticidade_2'].mean())

Margem Esperada Total Base Case: 770432.568237982
Margem Esperada Total Otimizada: 498203.81187089736
Retenção Média Base Case: 0.5073676747519267
Retenção Média Otimizada: 0.5026309974805719
Premio Medio Otimizado: 3009.1527695587147
Premio Medio Base Case: 2993.303855515689
Elasticidade: -1.6389693210790068
Elasticidade2: -1.6428786657757861


In [ ]:
df['Demanda_base']

In [ ]:
df['precos_otimizados'].mean()

In [ ]:
#df['margem_esperada_base_case'] = df['Demanda_base']*(df['premio']-df['Act_cost'])

In [ ]:
df['fat_teste3']=df['fat_teste2']+df['elasticidade']

In [ ]:
df['fat_teste3'].mean()

In [ ]:
print(df['IC_ofer_python_bin'].value_counts(ascending=False))

In [ ]:
df_t = df.query('chave == 288900056083100001')
df_t[["premio_anterior",'elasticidade','elasticidade_bin',"premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior_py","varia_prem_ofer_py"]]

In [ ]:
#df[['IC_ofer_python','fat_ajuste_otim',"Demanda_base","retencao_otimizada","elasticidade","premio_anterior","premio","precos_otimizados","Premium Parameter"]]

In [ ]:
df['fat_teste'].describe()

In [ ]:
print(df['IC_ofer_eranix'].value_counts(ascending=False))

In [ ]:
df['elasticidade_bin'].unique()

In [ ]:
df.groupby(["elasticidade_bin"])["retencao_otimizada","precos_otimizados"].describe().T

In [ ]:
df_ver.loc[1440:1459,["fat_teste0","fat_teste1","fat_teste",'retencao_otimizada','Demanda_base','precos_otimizados','premio','elasticidade']]

In [ ]:
(df['retencao_otimizada']/df['Demanda_base']-1) /(df['precos_otimizados']/df['premio']-1)

In [ ]:
df[["precos_otimizados","premio"]].describe().T

In [ ]:
df[["varia_prem_anterior","varia_prem_ofer","varia_prem_anterior_earnix","varia_prem_ofer_earnix"]].describe().T

In [ ]:
#ver = df.query('varia_prem_ofer >= 1.1')
#ver

In [ ]:
#df_lote_otim_atu.groupby(["Amostra"])["Demanda_base"].describe().T

In [ ]:
ver[["premio_anterior","premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior","varia_prem_ofer"]]

In [ ]:
#data =df[["IC_ofer_python","elasticidade"]]


In [ ]:
#df[['premio','retencao_base_esperado','preco_otimizado','retencao_otimizada']]

In [ ]:
#df#.to_csv(r'C:/Users/vcatelan/Documents/Scripts/dftocsv2.csv', sep='|', encoding='utf-8', header='true')
#df.to_excel(r'df.xlsx', sep='|', encoding='utf-8', header='true')

In [ ]:
 df['retencao_otimizada'].mean()

#criando a função

In [ ]:
df_relatorio_otim = df.copy()

#df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']*1
df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']*df['fat_ajuste_otim']
#df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']*df_relatorio_otim['fator_retencao']*df['fat_ajuste_otim']


df_relatorio_otim['batente_otim']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio'],2)
df_relatorio_otim['batente_otim_prem_ant']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio_anterior'],2)

df_relatorio_otim['retencao_otimizada'] = calcular_retencao(df_relatorio_otim['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)


df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"]*df_relatorio_otim["premio_prob"]

df_relatorio_otim["premio_anterior_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["premio_anterior"]

comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])

premio_anterior_ofer    = np.mean(df_relatorio_otim["premio_anterior"])
premio_anterior_w   = np.sum(df_relatorio_otim["premio_anterior_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])

df_relatorio_otim['var_prem_earnix_py'] = df_relatorio_otim['Premium Parameter']/df_relatorio_otim['precos_otimizados']

df_relatorio_otim['comis_valor_py'] = df_relatorio_otim["per_com_pct"]*df_relatorio_otim['precos_otimizados'] 
df_relatorio_otim['comis_valor_earnix'] =df_relatorio_otim["per_com_pct"]*df_relatorio_otim['Premium Parameter'] 
df_relatorio_otim['DA_Ofer'] = 0.2749


apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749


print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))

print('comis_ofertado" : %s'  % np.round(comis_ofertado*100,2))
print('comis_esperado" : %s'  % np.round(comis_esperado*100,2))

print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

print('premio_anterior_ofer" : %s'  %np.round( premio_anterior_ofer,2))
print('premio_anterior_w" : %s'  %np.round( premio_anterior_w,2))

In [ ]:
df_relatorio_otim['precos_otimizados'].mean()

In [ ]:
df_relatorio_otim['batente_otim_prem_ant'].describe()

In [ ]:
df_relatorio_otim['batente_otim'].describe()

In [ ]:
df_relatorio_otim['var_prem_earnix_py'].describe()

In [ ]:
df_t = df_relatorio_otim.query('chave == 288900056083100001')
df_t[["premio_anterior",'elasticidade','elasticidade_bin',"premio","precos_otimizados","Premium Parameter","LB","UB","varia_prem_anterior_py","varia_prem_ofer_py"]]

In [ ]:
df_export_analise_2 =df_relatorio_otim[["chave",'IC_ofer_eranix','IC_ofer_python_bin','elasticidade','elasticidade_bin',"Renovacao_ant","flag_idade_ult_rns",'comis_valor_py','comis_valor_earnix','DA_Ofer',"Act_cost","TARIFA","premio_anterior","per_com_pct","precos_otimizados","Premium Parameter","varia_prem_py_earnix","Demanda_base","retencao_otimizada",'Demanda_basecase',"Renewal Demand","IC_ofer_python","IC_ofer_eranix","varia_prem_anterior_py","varia_prem_ofer_py","varia_prem_anterior_earnix","varia_prem_ofer_earnix","SP_ofer_python","SP_ofer_eranix"]]
#df_export_analise_2.columns.to_list()

In [ ]:
df_t = df_export_analise_2.query('chave == 288900056083100001')
df_t[["premio_anterior",'elasticidade','elasticidade_bin',"precos_otimizados","Premium Parameter","varia_prem_anterior_py","varia_prem_ofer_py"]]

In [ ]:
df_export_analise_2.groupby(["elasticidade_bin"])["retencao_otimizada","precos_otimizados"].describe().T

In [ ]:
df_export_analise_2.to_csv(r'df_export_analise_3m.csv', sep='|' ,header='true')

In [ ]:
df.to_csv(r'analitico_202410_v1.csv', sep='|' ,header='true')

In [ ]:
df_relatorio_otim[['Demanda_base','retencao_otimizada','premio','preco_otimizado']]
, sep='|', encoding='utf-8', header='true'

In [ ]:
prices_df = pd.DataFrame({
    'chave': df_relatorio["chave"],
    'Act_cost':  df_relatorio["Act_cost"],
    'Premio ofertado com batente':  df_relatorio["Premio_oferecido_com_batente"],
    'Optimized Prices': df_relatorio["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_relatorio["prob_est"],
    'renewal demand':df_relatorio["demanda otimizada"],
    'variação':df_relatorio["premio"]/df_relatorio["Premio_oferecido_com_batente"]
})
prices_df.head(20)

In [ ]:
df_teste = df

In [ ]:
delta_q = (df_teste['retencao_otimizada']-df_teste['Demanda_base'])
delta_p = (df_teste['precos_otimizados']-df_teste['premio'])
   
delta_q_b = ((df_teste['retencao_otimizada']+df_teste['Demanda_base'])/2)
delta_p_b = ((df_teste['precos_otimizados']+df_teste['premio'])/2)

df_teste["elasticidade"] = (delta_q/delta_p)*(delta_p_b/delta_q_b)
df_teste["elasticidade_2"] = (df_teste['retencao_otimizada']/df_teste['Demanda_base']-1)/(df_teste['precos_otimizados']/df_teste['premio']-1)

df_teste.replace([np.inf, -np.inf], -1, inplace=True)
    
df_teste['elasticidade_bin'] = pd.cut(x=df_teste['elasticidade'], bins=[-3.5,-2.5,-2,-1.5,-1,0])  


df_teste["SP_ofer_python"] = df_teste["Act_cost"]/df_teste["precos_otimizados"]
df_teste["IC_ofer_python"] = df_teste["per_com_pct"]+ df_teste["SP_ofer_python"] +  0.2749
df_teste["SP_ofer_eranix"] = df_teste["Act_cost"]/df_teste["Premium Parameter"]
df_teste["IC_ofer_eranix"] = df_teste["SP_ofer_eranix"] + df_teste["per_com_pct"]+  0.2749
df_teste["varia_prem_anterior_py"] = df_teste["precos_otimizados"]/df_teste["premio_anterior"]
df_teste["varia_prem_anterior_earnix"] = df_teste["Premium Parameter"]/df_teste["premio_anterior"]
df_teste["varia_prem_ofer_py"] = df_teste["precos_otimizados"]/df_teste["premio"]
df_teste["varia_prem_ofer_earnix"] = df_teste["Premium Parameter"]/df_teste["premio"]
df_teste["varia_prem_py_earnix"] = df_teste["precos_otimizados"]/df_teste["Premium Parameter"]

df_teste['IC_ofer_python_bin'] = pd.cut(x=df_teste['IC_ofer_python'], bins=[0.7,0.95,0.99,1,1.03,1.9])  
df_teste['IC_ofer_eranix'] = pd.cut(x=df_teste['IC_ofer_eranix'], bins=[0.7,0.95,0.99,1,1.03,1.9]) 


In [ ]:
df['fat_ajuste_otim'] = 1.0  

for index, row in df.iterrows():
    if row['IC_ofer_python'] <= 0.95:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.96
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.96
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.98
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.97
            
    elif row['IC_ofer_python']<= 0.99:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.976
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.975
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.972
            
             
    elif row['IC_ofer_python']<= 1:
        if row['elasticidade']   <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 0.991
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 0.994
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 0.995
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 0.999           
            
            
            
    elif row['IC_ofer_python']<= 1.03:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1.014
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1.016
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1.013
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1.015
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.018              
            
    elif row['IC_ofer_python']<= 3:
        if row['elasticidade'] <= -2.5:
            df.at[index, 'fat_ajuste_otim'] = 1.026
        elif row['elasticidade'] <= -2.0:
            df.at[index, 'fat_ajuste_otim'] = 1.026
        elif row['elasticidade'] <= -1.5:
            df.at[index, 'fat_ajuste_otim'] = 1.026
        elif row['elasticidade'] <= -1:
            df.at[index, 'fat_ajuste_otim'] = 1.02
        elif row['elasticidade'] <= 0:
            df.at[index, 'fat_ajuste_otim'] = 1.04 